In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 


dataset = ['EFIFI','Nair'][1]
basedir = ['../../bf_data/EFIGI_catalog/','../../bf_data/Nair_and_Abraham_2010/'][1]
fitsdir = basedir + ['fits_temp_Jan_19/','fits_temp_Dec_28/', 'fits_temp_Feb_3/'][2]

out_dir = basedir+'out1/'
fns_r = glob(fitsdir+"*/*r.fits")
fns_r.sort()

# Generate mask + Initial StatMorph

In [2]:
fns_r

['../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000002.10p155254.15/J000002.00+155255.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000007.81m000226.09/J000007.00-000224.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000012.78p010712.72/J000012.00+010715.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000040.04p143307.72/J000040.00+143310.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000047.89p141639.09/J000048.00+141640.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000133.95p150453.64/J000134.00+150455.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000200.82p150132.23/J000200.00+150133.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000206.34m101306.79/J000206.00-101304.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000311.26p155756.61/J000311.00+155757.0-r.fits',
 '../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/J000321.63

In [3]:
from astrobf.utils import gen_mask

In [4]:
gen_mask.run_mask(fns_r[:1], out_dir, npix_min=40)

../../bf_data/Nair_and_Abraham_2010/out1/J000002.10p155254.15_mask.pickle
Overwrite? (Y:Yes N:No A:yes for All A


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt: 


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f72b06bdaf0> (for post_execute):


KeyboardInterrupt: 

So far, So good.

In [ ]:
from astrobf import measure_morph
# Run StatMorph for the first time
# Just in case, only for this first run, I pickle all 'morph' instances. a few 10s of GB in total.
measure_morph.run_stat_morph_init(fns[:10], out_dir, eps=1e-6)

# Visual inspection - Filter out bad samples
Keep good samples under `.../stat_png/good/*.png`

In [ ]:
all_morphs = glob(out_dir+"Morphs/final_morphs*.pickle")
all_morphs.sort()
for alm in all_morphs:
    print(alm)
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        img_name = morph._gid
        fn = get_fn(img_name)
            
        hdulist = fits.open(fn)
        img = hdulist[0].data
        img -= (img.min() - eps)
        
        image_diagnostics.make_figure(morph, nrows=3,
                                      savefig=gen_fn_img(morph), img_org=None)

In [165]:
good_list = glob(basedir+'out1/'+"stat_png/good/*.png")
good_gids=[]
for gg in good_list:
    good_gids.append(gg.split("/")[-1].split("_")[1])
good_gids.sort()
good_fns = [glob(fitsdir+gg+'/*r.fits')[0] for gg in good_gids]

#### Following part will be updated if I have to go through this step again.

In [ ]:
# Catalog of StatMorph properties of the good sample.
from astrobf.utils.measure_morph import statMorph_fields
fields = statMorph_fields[:6]
ngal = len(good_gids)
result_arr = np.zeros(ngal, 
                      dtype=[('id','<U24'),('ttype',int),('flag',bool),('flag_sersic',bool)]
                           +[(ff,float) for ff in fields])
i=0
for alm in all_morphs:
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        if morph._gid in good_gids:
            result_arr[i]['id'] = morph._gid
            for ff in fields:
                result_arr[i][ff] = getattr(morph, ff)
            i+=1
        else:
            pass

# Merge files to minimize I/O

In [57]:
from astropy.io import fits
import pickle
from astrobf.utils import mask_utils

all_data=[]
for fn in good_fns:
    img_name = fn.split("/")[-2]
    img = fits.open(fn_img)[0].data
    weight = fits.open(fn_weight)[0].data
    mask = pickle.load(open(out_dir+f"/masks/{img_name}_mask.pickle", 'rb'))
    slices = mask_utils._get_stamp_range(mask)
    data = np.stack((img[slices], mask[slices], weight[slices]))
    all_data.append({'data':data, 'img_name':img_name, 'slices':slices})

pickle.dump(all_data, open("all_gals.pickle", "wb"))

# t-types from catalog

Nair catalog has very specific and inconvenient format. You need a dedicated function for this.

In [168]:
cat = utils.misc.load_Nair('../../bf_data/Nair_and_Abraham_2010/catalog/table2.dat')

inds = cat['ID'].searchsorted(result_arr['id'])
print("Is every element matched?: ", np.all(cat[inds]['ID'] == result_arr['id']))
result_arr['ttype'] = cat[inds]['TT']

# Save the complete catalog
pickle.dump(result_arr, open(out_dir+'result_arr.pcikle', 'wb'))

AttributeError: module 'astrobf.utils' has no attribute 'misc'

# Initialize BO loop -  Measure morphology features

In [ ]:
import torch
import math

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

### Initial guesses 


# Run Bayesian Optimization loop